In [1]:
import sys
sys.path.append("..") # Ensure the parent directory is in the path

import core.assingment as Models
from core.DataLoader import DataPreprocessor, DataConfig
import core
import numpy as np
from importlib import reload
import keras
import matplotlib.pyplot as plt
MAX_JETS = 6
PLOTS_DIR = f"plots/"
import os
if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)

config = DataConfig(jet_features=["ordered_jet_pt", "ordered_jet_e", "ordered_jet_phi", "ordered_jet_eta", "ordered_jet_b_tag"], 
                                lepton_features=["lep_pt","lep_e", "lep_eta", "lep_phi"],
                                jet_truth_label="ordered_event_jet_truth_idx", 
                                lepton_truth_label="event_lepton_truth_idx", 
                                global_features = ["met_met_NOSYS","met_phi_NOSYS"], 
                                max_leptons=2, 
                                max_jets = MAX_JETS, 
                                non_training_features =["truth_ttbar_mass", "truth_ttbar_pt", "N_jets"], 
                                event_weight="weight_mc_NOSYS")

DataProcessor = DataPreprocessor(config)
DataProcessor.load_data("/data/dust/group/atlas/ttreco/full_training.root", "reco", max_events=1000000)
DataProcessor.normalise_data()
X_train,y_train, X_val, y_val = DataProcessor.split_data(test_size=0.1, random_state=42)

2025-10-02 12:56:08.335626: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-02 12:56:08.336483: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-02 12:56:08.340163: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-02 12:56:08.349354: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759402568.363948   43835 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759402568.36

In [ ]:
reload(Models)
reload(core)
TransformerMatcher = Models.CrossAttentionModel(config, name="Transformer")

TransformerMatcher.build_model(
    num_heads=8,
    hidden_dim=48,
    num_decoder_layers=4,
    num_encoder_layers=4,
    dropout_rate=0.1
)
TransformerMatcher.compile_model(
    loss = core.utils.AssignmentLoss(lambda_excl=0), optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-5), metrics=[core.utils.AssignmentAccuracy()]
)
TransformerMatcher.model.summary()
#TransformerMatcher.load_model("Transformer_Assignment.keras")

2025-10-02 12:56:51.974705: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "AssignmentTransformer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ global_inputs       │ (None, 1, 2)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 2)         │          0 │ global_inputs[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_inputs          │ (None, 6, 5)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector       │ (None, 6, 2)      │          0 │ flatten[0][0]     │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 6, 7)      │          0 │ jet_inputs[0][0], │
│ (Concatenate)       │                   │            │ repeat_vector[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_embedding (MLP) │ (None, 6, 48)     │      1,702 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_mask            │ (None, 6, 1)      │          0 │ jet_inputs[0][0]  │
│ (GenerateMask)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention_blo… │ (None, 6, 48)     │     18,960 │ jet_embedding[0]… │
│ (SelfAttentionBloc… │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention_blo… │ (None, 6, 48)     │     18,960 │ self_attention_b… │
│ (SelfAttentionBloc… │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_inputs          │ (None, 2, 4)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_1     │ (None, 2, 2)      │          0 │ flatten[0][0]     │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention_blo… │ (None, 6, 48)     │     18,960 │ self_attention_b… │
│ (SelfAttentionBloc… │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 2, 6)      │          0 │ lep_inputs[0][0], │
│ (Concatenate)       │                   │            │ repeat_vector_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention_blo… │ (None, 6, 48)     │     18,960 │ self_attention_b… │
│ (SelfAttentionBloc… │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_embedding (MLP) │ (None, 2, 48)     │      1,535 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leps_cross_attenti… │ (None, 2, 48)     │     19,056 │ lep_embedding[0]… │
│ (MultiHeadAttentio… │                   │            │ self_attention_b… │
│                     │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leps_ffn_0 (MLP)    │ (None, 2, 48)     │      4,704 │ leps_cross_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leps_cross_attenti… │ (None, 2, 48)     │     19,056 │ leps_ffn_0[0][0]

 Total params: 178,821 (698.52 KB)

 Trainable params: 178,821 (698.52 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:
TransformerMatcher.train_model(epochs=30,
                                X_train=X_train,
                                y_train=y_train,
                                batch_size=512,
                                callbacks = keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, restore_best_weights=True, mode ="max"))


Epoch 1/100
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 328s 219ms/step - assignment_accuracy: 0.1119 - loss: 0.8658 - val_assignment_accuracy: 0.1344 - val_loss: 0.7668
Epoch 2/100
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 304s 217ms/step - assignment_accuracy: 0.2312 - loss: 0.7678 - val_assignment_accuracy: 0.3469 - val_loss: 0.6830
Epoch 3/100
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 321s 229ms/step - assignment_accuracy: 0.3169 - loss: 0.7161 - val_assignment_accuracy: 0.3581 - val_loss: 0.6689
Epoch 4/100
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 359s 256ms/step - assignment_accuracy: 0.3346 - loss: 0.6955 - val_assignment_accuracy: 0.3624 - val_loss: 0.6599
Epoch 5/100
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 367s 261ms/step - assignment_accuracy: 0.3433 - loss: 0.6842 - val_assignment_accuracy: 0.3641 - val_loss: 0.6531
Epoch 6/100
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 321s 229ms/step - assignment_accuracy: 0.3501 - loss: 0.6757 - val_assignment_accuracy: 0.3574 - val_loss: 0.6522
Epoch 7/100
1403/1403 ━━━━━━━━━━━━━━━━━━━━ 347s 247ms/step

KeyboardInterrupt: 

In [ ]:
TransformerMatcher.predict_indices(X_val, batch_size=1024)

In [ ]:
y_val

In [ ]:
TransformerMatcher.save_model("Transformer_Assignment.keras")

In [ ]:
reload(core)
ml_eval = core.assingment.MLEvaluatorBase(TransformerMatcher, X_val, y_val)
importances = ml_eval.compute_permutation_importance(num_repeats=5)

In [ ]:
pred_val = TransformerMatcher.predict_indices(X_val, batch_size=1024)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_predictions(pred_val[:,:,0].argmax(axis=-1), y_val[:,:,0].argmax(axis=-1))


In [ ]:
ml_eval.plot_feature_importance(importances)